# Leaf Disease Detection

Learning goals:
- Learn how to create datasets with TensorFlow data API
- Learn how to train a DenseNet for leaf disease classification task
- Learn how to submit your predictions

In [ ]:
import os
from os.path import join

from tqdm.notebook import tqdm # progress bar

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow.keras.models import Model

import cv2

import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

from sklearn.model_selection import train_test_split

from IPython.display import SVG


## Load Data

In [ ]:
IMAGE_PATH = "../input/plant-pathology-2020-fgvc7/images/"
TEST_PATH = "../input/plant-pathology-2020-fgvc7/test.csv"
TRAIN_PATH = "../input/plant-pathology-2020-fgvc7/train.csv"
SUB_PATH = "../input/plant-pathology-2020-fgvc7/sample_submission.csv"
MODEL_PATH = "models/plant_pathology_model.h5"

sub = pd.read_csv(SUB_PATH)
df_test = pd.read_csv(TEST_PATH)
df_train = pd.read_csv(TRAIN_PATH)

EPOCHS = 50

# Define size of the image to train on
IMAGE_X = 224
IMAGE_Y = 224

labels = ['healthy', 'multiple_diseases', 'rust', 'scab']

# Explore the data

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
print('Training set size:', len(df_train))
for label in labels:
    print(f"\t{label}: {df_train[df_train[label]==1].shape[0]}")
print('Test set size:', len(df_test))

## Check if every image belongs to each class

In [ ]:
# Sum all of the labels together
df_train['number_of_classes'] = df_train['healthy'] + \
                                df_train['multiple_diseases'] + \
                                df_train['rust'] + df_train['scab']

# mean should be 1, std should be 0
df_train['number_of_classes'].mean(), df_train['number_of_classes'].std()

In [ ]:
def load_image(image_id):
    file_path = "{}{}".format(image_id, ".jpg")
    image = cv2.imread(join(IMAGE_PATH, file_path))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

# plot some images
def plot_sample_images(preprocess_fn=None, nrows=4, ncols=4):
    fig, axs = plt.subplots(nrows, ncols,
                            figsize=(12,10)
                           )
    axs = axs.ravel() # make 1D array for easy plotting in for loop

    for i, image_id in enumerate(np.random.randint(len(df_train), size=nrows*ncols)):
        # show an image
        img = load_image(df_train['image_id'][image_id])
        
        if preprocess_fn:
            img = preprocess_fn(img)
        axs[i].imshow(img)
        axs[i].axis(False)
        label = df_train.loc[:, 'healthy':].iloc[image_id, :].idxmax()
        axs[i].set_title('{} | {}'.format(df_train['image_id'][image_id], label))
        plt.tight_layout()
plot_sample_images(nrows=4, ncols=4)

In the plot below you can check the exact values of each pixel. Notice that most of the pixels have high green and low blue values. However the spot on the leaf has high blue value.

In [ ]:
image = load_image(df_train['image_id'][0])
fig = px.imshow(image)
fig.show()

Lets have a closer look at the distribution of the channel values.

In [ ]:
# Code thanks to https://www.kaggle.com/tarunpaparaju/plant-pathology-2020-eda-models

train_images = df_train["image_id"][:100].apply(load_image)

red_values = [np.mean(train_images[idx][:, :, 0]) for idx in range(len(train_images))]
green_values = [np.mean(train_images[idx][:, :, 1]) for idx in range(len(train_images))]
blue_values = [np.mean(train_images[idx][:, :, 2]) for idx in range(len(train_images))]
values = [np.mean(train_images[idx]) for idx in range(len(train_images))]

fig = ff.create_distplot([red_values, green_values, blue_values],
                         group_labels=["R", "G", "B"],
                         colors=["red", "green", "blue"])
fig.update_layout(title_text="Distribution of channel values")
fig

From the plot above we can see that the most pronounced color is green, which has the highest values. All other channels are shifted to the left with blue being the least pronounced.

## Data Generators and Augmentation
For loading and augmenting the data we are going to use the tensorflow dataset API.

In [ ]:
# Convert the names of the images into a correct path
def format_path(st):
    return os.path.join(IMAGE_PATH, st + '.jpg')

# Genreate train and test paths
train_paths = df_train.image_id.apply(format_path).values
test_paths = df_test.image_id.apply(format_path).values

# Convert the labels to floats
train_labels = np.float32(df_train.loc[:, 'healthy':'scab'].values)
# Split the data into validation and training sets
train_paths, valid_paths, train_labels, valid_labels = train_test_split(train_paths, train_labels, test_size=0.15, random_state=2020)

In [ ]:
def decode_image(filename, label=None, image_size=(IMAGE_X, IMAGE_Y)):
    """
    Loads, normalizes and resizes the image
    """
    bits = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(bits, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, image_size)
    
    if label is None:
        return image
    else:
        return image, label

def data_augment(image, label=None):
    """
    Define your data augmentations here
    """
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    
    if label is None:
        return image
    else:
        return image, label

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 64

train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((train_paths, train_labels))
    .map(decode_image, num_parallel_calls=AUTO)
    .map(data_augment, num_parallel_calls=AUTO)
    .repeat()
    .shuffle(512)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((valid_paths, valid_labels))
    .map(decode_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(test_paths)
    .map(decode_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
)

## DenseNet
For this example we are going to use DenseNet model with pretrained weights. Instead of relying on extremely deep architectures DenseNet encourages feature reuse. The output of each Dense Block is being used as input for the next one.
<div align="center">
<a><img src="https://files.ai-pool.com/m/densenet.png" width="600"></a>
</div>

More details can be found in the lecture slides or in the [original paper](https://arxiv.org/pdf/1608.06993.pdf).

Let's train it on the leaf disease detection task and evaluate its performance.

### Model Definition

In [ ]:
from tensorflow.keras.applications import MobileNet, DenseNet121
from tensorflow.keras import layers

# Define model architecture
model = tf.keras.Sequential(
    [DenseNet121(
        input_shape=(IMAGE_X, IMAGE_Y, 3),
        weights='imagenet',
        include_top=False),
     layers.GlobalAveragePooling2D(),
     layers.Dense(train_labels.shape[1], activation='softmax')
    ]
)
        
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])
model.summary()

### Callbacks

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Define checkpointing callback
mcp = ModelCheckpoint(MODEL_PATH, monitor='val_loss', save_best_only=True, verbose=0)

# Define learnning rate schedule
def build_lrfn(lr_start=0.00001, lr_max=0.00005, 
               lr_min=0.00001, lr_rampup_epochs=5, 
               lr_sustain_epochs=0, lr_exp_decay=.8):
    lr_max = lr_max

    def lrfn(epoch):
        if epoch < lr_rampup_epochs:
            lr = (lr_max - lr_start) / lr_rampup_epochs * epoch + lr_start
        elif epoch < lr_rampup_epochs + lr_sustain_epochs:
            lr = lr_max
        else:
            lr = (lr_max - lr_min) *\
                 lr_exp_decay**(epoch - lr_rampup_epochs\
                                - lr_sustain_epochs) + lr_min
        return lr
    return lrfn

# Create a learning rate schedule as keras callback
lrfn = build_lrfn()
lr_schedule = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=1)

In [ ]:
# Visualize the learning rate across epochs
epochs_dummy = list(range(0, 50))
y = [lrfn(e) for e in epochs_dummy]
fig = go.Figure(go.Scatter(x=epochs_dummy, y=y, mode='lines+markers'))
fig.update_layout(
    yaxis = dict(
        showexponent='all',
        exponentformat='e'
    ),
    title='Learning rate schedule'
)

### Training

In [ ]:
callbacks = [
    lr_schedule, # use learning rate scheduler
    mcp,         # checkpoint models
]

history = model.fit(train_dataset,
                    epochs=EPOCHS,
                    callbacks=callbacks,
                    steps_per_epoch=len(df_train) // BATCH_SIZE,
                    validation_data=valid_dataset)

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots


def visualize_training_process(history):
    """ 
    Visualize loss and accuracy from training history
    
    :param history: A Keras History object
    """
    history_df = pd.DataFrame(history.history)
    epochs = np.arange(1, len(history_df) + 1)
    fig = make_subplots(2, 1)
    fig.append_trace(go.Scatter(x=epochs, y=history_df['categorical_accuracy'], mode='lines+markers', name='Accuracy Train'), row=1, col=1)
    fig.append_trace(go.Scatter(x=epochs, y=history_df['val_categorical_accuracy'], mode='lines+markers', name='Accuracy Val'), row=1, col=1)
    
    fig.append_trace(go.Scatter(x=epochs, y=history_df['loss'], mode='lines+markers', name='Loss Train'), row=2, col=1)
    fig.append_trace(go.Scatter(x=epochs, y=history_df['val_loss'], mode='lines+markers', name='Loss Val'), row=2, col=1)
    
    fig.update_layout( xaxis_title="Epochs", template="plotly_white")
    
    return fig
visualize_training_process(history)

In [ ]:
# Evaluate performance of model by plotting confusion matrix
from sklearn.metrics import confusion_matrix

# see http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
import itertools

def accuracy(y, y_pred):
    return np.sum(y == y_pred)/len(y)

def plot_confusion_matrix(cm, labels=None, title='Confusion Matrix'):
    import plotly.figure_factory as ff

    x = labels
    y = x

    # change each element of z to type string for annotations
    z_text = [[str(y) for y in x] for x in cm]

    # set up figure 
    fig = ff.create_annotated_heatmap(cm, x=x, y=y, annotation_text=z_text, colorscale='YlGnBu', showscale=True)

    # add title
    fig.update_layout(title_text=title,
                      #xaxis = dict(title='x'),
                      #yaxis = dict(title='x')
                     )

    # add custom xaxis title
    fig.add_annotation(dict(font=dict(color="black",size=14),
                            x=0.5,
                            y=-0.15,
                            showarrow=False,
                            text="Predicted value",
                            xref="paper",
                            yref="paper"))

    # add custom yaxis title
    fig.add_annotation(dict(font=dict(color="black",size=14),
                            x=-0.35,
                            y=0.5,
                            showarrow=False,
                            text="Real value",
                            textangle=-90,
                            xref="paper",
                            yref="paper"))

    # adjust margins to make room for yaxis title
    fig.update_layout(margin=dict(t=100, l=200), width=700, height=600)
    fig.show()
    
# predict labels from validation set
y_pred = model.predict(valid_dataset)
# convert data to label number
y_pred = np.argmax(y_pred, axis=1) 
y_true = np.argmax(valid_labels, axis=1) 

# compute the confusion matrix
cm = confusion_matrix(y_true, y_pred) 

plot_confusion_matrix(cm, labels, title='Confusion_matrix Validation Set (acc={:.3f})'.format(accuracy(y_true, y_pred)))

# Save predictions

In [ ]:
# Predict labels on the test set
predictions = model.predict(test_dataset)

# Prepare the submission file
sub.loc[:, 'healthy':] = predictions
sub.to_csv('submission_densenet.csv', index=False)
sub.head()